In [5]:
import os, sys
sys.path.append("../../")

from src.core.module import Module, Linear, LayerNorm
from src.core.losses import CrossEntropy, BCE
from src.core.optim import Standard
from src.core.tensor import Tensor
import numpy as np
from src.models.transformer import Transformer, Embedding
import time
from typing import List
from src.tokenizer.tokenizer import Tokenizer
import tqdm
import pandas as pd
import string
import re
import unicodedata
from src.tokenizer.tokenizer import Tokenizer


FileNotFoundError: [Errno 2] No such file or directory: 'src/tokenizer/token_to_id.json'

In [ ]:
df = pd.read_parquet(r"data\wikitext\wikitext-103-raw-v1\train-00000-of-00002.parquet")
df = pd.read_parquet(r"data\wikitext\wikitext-103-raw-v1\train-00000-of-00002.parquet")

df = df[df['text'].str.strip() != ""].reset_index(drop=True)
df = df[~df['text'].str.startswith(" =")].reset_index(drop=True)

In [4]:
allowed_chars = set(string.ascii_letters + string.digits + string.punctuation + " \n\t")

def clean_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")
    text = "".join(c for c in text if c in allowed_chars)
    text = text.replace(" @-@ ", "-")
    text = text.replace(" @,@ ", ",")
    text = re.sub(r"\s+([.,!?;':)\]])", r"\1", text)
    text = re.sub(r"([.,!?;:])([^\s])", r"\1 \2", text)
    text = re.sub(r"([[(])\s+", r"\1", text)
    text = text.replace("\'", "'")
    text = text.strip()
    return text

df['text'] = df['text'].apply(clean_text)

C:\Users\luequ\AppData\Local\Temp\ipykernel_19204\2570187245.py:11: FutureWarning: Possible nested set at position 2
  text = re.sub(r"([[(])\s+", r"\1", text)


In [5]:
df = df.sample(frac=1).reset_index(drop=True)
df['text'].iloc[12]

'In 2003, Chia mooted the idea that Singaporeans should be able to borrow from their own Central Provident Fund (CPF) savings to tide over periods of financial difficulty. His rationale was that if individuals could borrow from their own CPF accounts to buy depreciating property and money-losing stocks, there was no reason why they should not be allowed to do so to pay for bills. Dr. Ong Seh Hong, MP for Aljunied GRC, opposed the view, stating that it was important and for the good of Singapore that Singaporeans were independent individuals who could assume the risks and successes of their investments and be self-sufficient.'

In [6]:
df.to_parquet("data/data_cleaned.parquet")

In [14]:
train = pd.read_parquet("../../data/clean/train.parquet")
test = pd.read_parquet("../../data/clean/test.parquet")
validation = pd.read_parquet("../../data/clean/validation.parquet")

In [15]:
tokenizer = Tokenizer(token_to_id_path="../tokenizer/token_to_id.json", merges_path="../tokenizer/merges.json")

Tokenizer loaded with 21779 tokens


In [16]:
train.head()

,text,bin
0,Rupert was portrayed by Timothy Dalton in the ...,4
1,"In 1894, the peninsula was visited by the Russ...",32
2,"Humans play a key role in the biosphere, with ...",64
3,The 1990 Toledo Rockets football team represen...,24
4,"Samuel Noel "" Sam "" Strachan is a fictional ch...",38


In [17]:
validation['seq'] = validation['text'].apply(tokenizer.encode)
validation.to_parquet("../../data/clean/validation.parquet")

KeyboardInterrupt: 

In [ ]:
test['seq'] = test['text'].apply(tokenizer.encode)
test.to_parquet("../../data/clean/test.parquet")


In [ ]:
train['seq'] = train['text'].apply(tokenizer.encode)
train.to_parquet("../../data/clean/train.parquet")

KeyboardInterrupt: 